# 1. Load the libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import cv2
import glob
import numpy as np
import pandas as pd  
from PIL import Image
from skimage import measure
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import tensorflow as tf
from skimage.measure import label, regionprops, regionprops_table

2024-11-21 15:38:54.635666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 15:38:55.581247: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 2. EfficientNetB0

## 2.1 Load the model

In [2]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input

with tf.device('/CPU:0'):
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(50, 50, 3))
    layer = base_model.get_layer(name="top_conv").output
    output = GlobalAveragePooling2D()(layer)
    feature_extraction_model = Model(inputs=base_model.inputs, outputs=output)



2024-11-21 15:38:57.270625: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9612 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5


In [3]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'

#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

ch1 = [] #fibroblast
ch2 = [] #cancer
ch4 = [] #dapi


for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+2]) #for MHB ch3 is cancer 
    CH4_path = os.path.join(image_path, images[i+1]) #for MHB ch2 is DAPI

 
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1.append(np.array(ch1_img))
    ch2.append(np.array(ch2_img))
    ch4.append(np.array(ch4_img))

ch1_max = np.max(ch1)
ch2_max = np.max(ch2)
ch4_max = np.max(ch4)
print("Maximum intensity for channel 1:", ch1_max)
print("Maximum intensity for channel 2:", ch2_max)
print("Maximum intensity for channel 4:", ch4_max)

ch1_q099 =  np.quantile(ch1, 0.99)
ch2_q099 =  np.quantile(ch2, 0.99)
ch4_q099 =  np.quantile(ch4, 0.99)
print("Quantile_099 channel 1:", ch1_q099)
print("Quantile_099 channel 2:", ch2_q099)
print("Quantile_099 channel 4:", ch4_q099)

Maximum intensity for channel 1: 65289
Maximum intensity for channel 2: 65535
Maximum intensity for channel 4: 64458
Quantile_099 channel 1: 1959.0
Quantile_099 channel 2: 2514.0
Quantile_099 channel 4: 5185.0


## 2.3 Read images as tensors and extract features

In [ ]:
# Example of preprocessing and extracting features
def preprocess_image(image):
    image = tf.image.resize(image, (50, 50))  # Resize to match model input
    image = preprocess_input(image)  # Apply EfficientNetB0-specific preprocessing
    return image

#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'

#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

# "mean_feature_list" and "j" variables are initialized to empty list and 0, respectively to store the features for each image and to iterate over the list of masks.
mean_fetaure_list =[]
image_list = []
j=0

#loop with step size of 3 to iterate over a range of indices, pulling the filenames for three channels of each image.
for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+2])
    CH4_path = os.path.join(image_path, images[i+1])
    
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1_img_arr = np.array(ch1_img)
    ch2_img_arr = np.array(ch2_img)
    ch4_img_arr = np.array(ch4_img)
    

    #use cellpose mask to extract individual cell regions,to generate 50x50 pixel images.
    cellpose_path = os.path.join(mask_path, cellpose_mask[j])

    cellpose_img = Image.open(cellpose_path)
    cellpose_img_array = np.array(cellpose_img)
    labels = label(cellpose_img_array)
    regions = measure.regionprops(labels)

    j=j+1
    single_cell_bbox = []

    for props in regions:
        y0, x0 = props.centroid
        y = int(round(y0))
        x = int(round(x0))
        box_ch1 = ch1_img_arr[x-25:x+25,y-25:y+25]
        box_ch2 = ch2_img_arr[x-25:x+25,y-25:y+25]
        box_ch4 = ch4_img_arr[x-25:x+25,y-25:y+25]

        if (box_ch1.shape == (50,50)):
            coordinate = x, y
            
            box_ch1 = (box_ch1.astype('float32')/ch1_q099)*255
            box_ch2 = (box_ch2.astype('float32')/ch2_q099)*255
            box_ch4 = (box_ch4.astype('float32')/ch4_q099)*255
            
            singlecell = np.stack((box_ch1, box_ch2, box_ch4), axis=-1)
            reshaped_singlecell = singlecell.reshape((1,50,50,3))
            single_cell_bbox.append(reshaped_singlecell)

    single_cells = np.array(single_cell_bbox)

    if len(single_cells) != 0:
        
        print(images[i+2])
    
        #convert single cell objects into a tensor using tf.data.Dataset.from_tensor_slices.
        single_cell_tensor = tf.data.Dataset.from_tensor_slices(single_cells)

        def extract_features(image):
            preprocessed_image = preprocess_image(image)
            features = feature_extraction_model(preprocessed_image)
            return features

        #A map operation is used to apply a feature extraction function, extract_features, to each cell image 
        #using a pre-trained model. This creates a feature_dataset of extracted features.
        feature_dataset = single_cell_tensor.map(extract_features)

        #features are summed across all cells, divided by the number of cells, and added to the mean_feature_list.
        sum_feature_dataset = tf.zeros((1,1280))

        for f in feature_dataset:
            sum_feature_dataset += f

        mean_feature = sum_feature_dataset/len(feature_dataset)

        mean_fetaure_list.append(mean_feature)
        image_list.append(images[i])

r01c01ch3.tiff


/home/gauss/miniconda3/envs/tf24/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
2024-11-21 15:42:00.812143: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r01c02ch3.tiff


2024-11-21 15:42:04.473836: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r01c03ch3.tiff


...

r03c10ch3.tiff


2024-11-21 15:58:46.909746: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r03c11ch3.tiff


2024-11-21 15:58:50.228466: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r03c12ch3.tiff


In [ ]:
#Convert mean feature tensor array to numpy array
feature_list = [tensor.numpy() for tensor in mean_fetaure_list]

In [9]:
print(len(feature_list))
print(len(image_list))

382
382


In [10]:
# Create a 2D numpy array from the feature list
feature_array = np.vstack(feature_list)

# Create a DataFrame with column names
df = pd.DataFrame(feature_array, columns=['feature{}'.format(i+1) for i in range(1280)])

#add col to the beginning of the dataframe
df.insert(0, 'Image', image_list)
df

,Image,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1271,feature1272,feature1273,feature1274,feature1275,feature1276,feature1277,feature1278,feature1279,feature1280
0,r01c01ch1.tiff,-0.326331,-0.249747,0.803696,0.932714,3.003610,1.319327,1.096321,0.693639,1.019925,...,-0.399666,1.646056,-1.003649,2.598372,-2.402919,2.224718,0.917154,-0.537331,0.876763,0.108833
1,r01c02ch1.tiff,-0.427253,-1.608012,0.193798,0.371952,1.913906,0.318760,0.319078,-0.580244,2.257953,...,-1.475396,0.693453,-0.979301,1.719249,-2.453255,2.873041,1.218501,0.285302,-0.120919,-0.727252
2,r01c03ch1.tiff,-0.202621,-0.242851,0.274222,0.369905,2.450461,1.065330,1.786448,0.366083,1.740010,...,-0.216927,1.587973,-0.812825,2.066200,-2.449465,2.290055,0.302557,-0.358150,0.783385,0.249795
3,r01c04ch1.tiff,-0.062942,-1.403147,0.576722,0.382180,1.963891,0.159910,0.418672,-0.388481,2.028390,...,-1.361755,0.766916,-0.935756,1.777401,-1.738962,2.830278,1.068198,0.172767,0.225356,-0.189269
4,r01c05ch1.tiff,0.006196,-1.751421,0.758767,0.798562,1.977720,-0.291776,1.008913,-0.945463,2.233450,...,-1.839867,-0.357430,-1.595761,2.148508,-2.047678,3.590540,1.190888,-0.295476,0.119775,-0.671449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,r16c19ch1.tiff,-0.373001,-0.949434,0.444021,0.443753,2.269789,0.523914,0.706971,-0.035484,1.822780,...,-0.790623,1.145983,-0.757819,2.130203,-2.430788,2.717015,1.091516,0.104924,0.361756,-0.595484
378,r16c20ch1.tiff,-0.314785,-0.944300,0.317680,0.537033,2.184880,0.552374,0.925716,-0.019278,1.762107,...,-0.487176,1.404500,-0.730524,2.171973,-2.099144,2.634290,1.005984,-0.069599,0.441732,-0.325545
379,r16c21ch1.tiff,-0.487845,0.231393,0.833350,1.001417,3.412989,1.842001,1.853675,1.099116,0.703236,...,0.189063,1.933399,-0.840025,2.505893,-2.542350,2.089836,0.511435,-0.928876,0.991759,0.812149
380,r16c22ch1.tiff,-0.404531,0.208486,0.878521,0.979772,3.478022,1.696777,1.575049,1.144019,0.706331,...,0.136010,1.849847,-0.788650,2.623852,-2.328361,2.117637,0.740555,-0.849657,1.052107,0.789867


In [ ]:
#save dataframe as excel 
df.to_excel("/home/gauss/Desktop/Revised_Data_Features/Pretrained baseline EfficientNetB0 Bbox Size 50/MHB/Plate_7.xlsx")
